# 시계열 데이터의 Anomaly Detection 방법 비교

- RNN을 이용한 지도학습 방법 - 이진 분류
- Autoencoder 사용 방법 - 재구성 오류 측정
- Variational Autoencoder 사용 방법 - 재구성 오류 측정

- 이 연습문제에서는 RNN, 자동 인코더 및 변형 자동 인코더를 사용하여 [ECG5000 데이터 세트](http://www.timeseriesclassification.com/description.php?Dataset=ECG5000)에서 이상을 감지합니다.  이 데이터 세트에는 각각 140개의 데이터 포인트가 있는 5,000개의 심전도가 포함되어 있습니다. 각 예제에 0(비정상 리듬에 해당) 또는 1(정상 리듬에 해당)로 레이블이 지정된 데이터 세트의 단순화된 버전을 사용할 것입니다. 우리는 비정상적인 리듬을 식별하는 데 관심이 있습니다.   

- "ECG5000" 데이터셋은 Physionet에서 다운로드한 20시간 길이의 ECG 데이터로, BIDMC 심부전 데이터베이스(chfdb)의 "chf07" 기록입니다. 이 데이터는 원래 **Goldberger**와 여러 연구자들이 참여한 논문 "PhysioBank, PhysioToolkit, and PhysioNet: 복잡한 생리학적 신호를 위한 새로운 연구 자원의 구성 요소"에서 발표되었습니다. 데이터 전처리는 두 가지 단계로 이루어졌습니다: (1) 각 심박을 추출하고, (2) 각 심박의 길이를 보간(interpolation)으로 동일하게 맞췄습니다. 이 데이터셋은 원래 **"A general framework for never-ending learning from time series streams"** 논문(DAMI 29(6))에서 사용되었으며, 이후 5,000개의 심박이 무작위로 선택되었습니다. 환자는 심각한 울혈성 심부전을 앓고 있었으며, 클래스 값은 자동으로 주석이 추가되었습니다.

## Load and prepare ECG data

In [ ]:
# Dataset Download

In [ ]:
# 데이터프레임의 마지막 열에 해당하는 레이블(정답)을 가져옵니다.
# 심전도 데이터의 특성(feature) 값들을 가져옵니다.

normal, abnormal data 분리

In [ ]:
# y_train이 True인 경우, 즉 정상 데이터에 해당하는 X_train, X_test, y_train, y_test 데이터를 각각 추출
# y_train이 False인 경우, 즉 비정상 데이터에 해당하는 X_train, X_test, y_train, y_test 데이터를 각각 추출
# 정상 데이터와 비정상 데이터의 shape 출력

In [ ]:
# 정상 학습 데이터를 TensorFlow Dataset으로 변환하고, 무작위로 섞은 후 배치 크기 128로 분할
# 정상 테스트 데이터를 TensorFlow Dataset으로 변환하고, 배치 크기 128로 분할

정상 ECG 시각화

In [ ]:
# 한 줄에 10개의 서브플롯을 생성합니다.
# 각 서브플롯에 대해 ECG 데이터를 플롯합니다.

비정상 ECG 시각화

In [ ]:
# 각 서브플롯에 대해 ECG 데이터를 플롯합니다.

### 이상 탐지를 위한 RNN
데이터 세트의 레이블에 액세스할 수 있으므로 이상 감지를 이진 분류 지도 학습 문제로 설정할 수 있습니다.

In [ ]:
# RNN 모델 입력에 맞게 X_train 데이터에 새로운 차원을 추가하여 3D 텐서로 변환
# RNN 모델 입력에 맞게 X_test 데이터에 새로운 차원을 추가하여 3D 텐서로 변환
# 변환된 X_train_rnn과 X_test_rnn의 shape 출력

In [ ]:
# RNN 모델 생성

In [ ]:
# Test set 정상/비정상 분류
# metric 계산

### 이상 감지를 위한 오토인코더
autoencoder의 목적은 주어진 입력의 재구성 오류를 최소화하는 것입니다. 따라서 우리는 최소한의 오류로 이러한 예를 재구성할 수 있도록 정상 심전도 시퀀스에 대해서만 자동 인코더를 훈련할 것입니다.   
비정상적인 리듬은 정상 시퀀스보다 재구성 오류가 더 높아야 하므로 재구성 오류가 고정 임계값보다 높은 경우의 리듬을 이상으로 분류할 수 있습니다.

In [ ]:
class AnomalyDetectorAE(Model):
    def __init__(self):
        # encoder network
        # decoder network
    def call(self, x):

오토인코더는 정상 ECG만 사용하여 훈련됩니다.

정상 데이터를 오토인코더로 인코딩과 디코딩을 한 후의 신호를 살펴봅니다.

In [ ]:
# 오토인코더의 인코더를 사용하여 정상 테스트 데이터(normal_X_test)를 인코딩하여 잠재 공간 표현(encoded_signals) 생성
# 오토인코더의 디코더를 사용하여 인코딩된 신호(encoded_signals)를 복원(decoded_signals)하여 원본 형태로 변환
# 첫 번째 테스트 샘플의 원본 신호(normal_X_test[0])를 파란색 선으로 플롯
# 첫 번째 테스트 샘플의 복원된 신호(decoded_signals[0])를 빨간색 선으로 플롯
# 원본 신호와 복원된 신호 사이의 차이를 시각화하기 위해 채워진 영역을 추가 (오차 시각화)
# 범례 추가

비정상적 데이터 예제에 대한 유사한 플롯 만들기:

In [ ]:
# 오토인코더의 인코더를 사용하여 비정상 테스트 데이터(abnormal_X_test)를 인코딩하여 잠재 공간 표현(encoded_signals) 생성
# 오토인코더의 디코더를 사용하여 인코딩된 신호(encoded_signals)를 복원(decoded_signals)하여 원본 형태로 변환
# 첫 번째 비정상 테스트 샘플의 원본 신호(abnormal_X_test[0])를 파란색 선으로 플롯
# 첫 번째 비정상 테스트 샘플의 복원된 신호(decoded_signals[0])를 빨간색 선으로 플롯
# 원본 신호와 복원된 신호 사이의 차이를 시각화하기 위해 채워진 영역을 추가 (오차 시각화)
# 범례 추가

### 이상 감지
재구성 손실이 고정 임계값보다 큰지 여부를 계산하여 이상을 감지합니다. 이를 위해 훈련 세트의 정상 예제에 대한 평균 평균 오류를 계산한 다음 재구성 오류가 훈련 세트의 표준 편차보다 높으면 미래 예제를 비정상적인 것으로 분류합니다.

훈련 세트의 정상 ECG에 대한 재구성 오류를 시각화 합니다.

In [ ]:
# 오토인코더 모델을 사용하여 정상 학습 데이터(normal_X_train)에 대한 복원 결과(reconstructions) 생성
# 정상 학습 데이터와 복원된 데이터 간의 MAE(Mean Absolute Error) 손실 계산
# 학습 손실(train_loss)의 분포를 히스토그램으로 시각화

평균 + 1 표준편차인 임계값을 선택합니다.

In [ ]:
# 학습 손실(train_loss)의 평균과 표준편차를 이용하여 이상 탐지에 사용할 임계값(threshold_ae) 설정
# 설정된 임계값 출력

테스트 세트의 대부분의 비정상 example은 임계값보다 재구성 오류가 더 큽니다. 임계값을 변경하여 분류기의 정밀도와 재현율을 조정할 수 있습니다.

In [ ]:
# 오토인코더 모델을 사용하여 비정상 학습 데이터(abnormal_X_train)에 대한 복원 결과(reconstructions) 생성
# 비정상 학습 데이터와 복원된 데이터 간의 MAE(Mean Absolute Error) 손실 계산
# 비정상 데이터의 복원 오차(test_loss) 분포를 히스토그램으로 시각화
# 설정된 임계값(threshold_ae)을 빨간색 선으로 표시
# 그래프 제목 및 축 레이블 설정
# 그래프 표시

재구성 오류가 임계값보다 큰 경우 ECG를 이상으로 분류합니다.  
재구성 오류를 이용한 이상치 예측과 label 을 비교하여 performance 를 측정합니다.

In [ ]:
# 모델, 데이터 및 임계값을 입력으로 받아서 예측을 수행하는 함수
def predict(model, data, threshold):
    # 주어진 모델로 데이터를 복원하고, 원본 데이터와 복원 데이터 간의 MAE 손실 계산
    # 손실이 임계값(threshold)보다 작은 경우 True, 크면 False 반환 (정상/비정상 판별)
# 예측 결과(predictions)와 실제 라벨(labels)을 입력으로 받아서 정확도, 정밀도, 재현율을 출력하는 함수
def print_stats(predictions, labels):
    # 정확도(Accuracy) 출력
    # 정밀도(Precision) 출력
    # 재현율(Recall) 출력

### 이상 감지를 위한 Variational Autoencoder
오토인코더는 훈련 데이터에 과적합되는 경향이 강합니다. 이 문제를 완화하도록 설계된 VAE를 사용합니다.

### Sampling Class

먼저 'Sampling' 클래스를 빌드합니다. 이것은 인코더 출력의 평균 (mu) 및 표준 편차 (sigma)와 함께 가우스 노이즈 입력을 제공하는 맞춤형 Keras 레이어입니다. 실제로 이 레이어의 출력은 다음 방정식으로 제공됩니다.

$$z = \mu + e^{0.5\sigma} * \epsilon  $$

여기서 $\mu$ = mean, $\sigma$ = standard deviation, $\epsilon$ = random sample

In [ ]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        # 인코더의 출력을 분해합니다.
        # 배치의 크기와 차원을 얻습니다.
        # 무작위 텐서를 생성합니다.
        # 재매개변수화 기법을 적용합니다.

### Kullback–Leibler Divergence
모델의 생성 능력을 향상 시키려면 잠재 공간에 도입된 랜덤 정규 분포를 고려해야 합니다. 이를 위해 [Kullback–Leibler Divergence](https://arxiv.org/abs/2002.07514)가 계산되어 재구성 손실에 추가됩니다. 공식은 아래 함수에서 정의됩니다.

In [ ]:
def kl_reconstruction_loss(inputs, outputs, mu, sigma):
    # KLD 계산 공식 적용
    # 배치 내 평균을 계산하고 -0.5를 곱하여 최종 KLD 손실을 계산

이제 전체 VAE 모델을 정의할 수 있습니다. KL reconstruction loss를 추가하기 위해 `model.add_loss()`를 사용합니다. 이 손실을 계산하는 것은 `y_true`와 `y_pred`를 사용하지 않으므로 `model.compile()`에서 사용할 수 없습니다.

- add_loss() 메서드 : 손실이 있는 경우, 자동으로 합산되어 주 손실에 추가

In [ ]:
# Encoder 정의
# Decoder 정의

### 변이형 오토인코더(VAE, Variational Autoencoder)를 구축하고 학습하기 위해 필요한 모델을 정의

In [ ]:
# 인코더를 통과시켜 평균(mu), 표준 편차(sigma), 그리고 잠재 공간의 벡터(z)를 얻습니다.
# 디코더를 사용하여 잠재 공간의 벡터(z)로부터 입력을 재구성
# 입력과 재구성된 값을 사용하여 VAE 모델을 생성
# Kullback-Leibler 발산을 계산하는 함수를 사용하여 손실을 추가합니다. 이는 잠재 공간의 분포가
# 우리가 원하는 분포(예: 표준 정규 분포)에 가까워지도록 돕습니다.
# add_loss는 사용자 정의 손실을 추가하는 메서드

In [ ]:
# 학습 과정에서의 평균 손실을 추적
# 평균 제곱 오차 손실 함수를 사용합니다.

In [ ]:
def train_step(x_batch_train):
        # VAE 모델에 배치 입력을 공급하여 재구성된 결과를 얻음
        # 재구성 손실을 계산
        # KLD 정규화 손실을 추가
    # 그라디언트를 계산하고 가중치 업데이트
# 테스트 스텝 추가
def test_step(x_batch_test):
# 손실 지표를 두 개로 분리
    # 에포크 시작 시 지표 리셋
        # 훈련 스텝
        # 테스트 스텝
        # 매 500 스텝마다 결과 표시

## Reconstruction Error 시각화

In [ ]:
# VAE(Variational Autoencoder) 모델을 사용하여 테스트 데이터(X_test)의 복원된 이미지를 예측
# 첫 번째 테스트 샘플의 원본 데이터(normal_X_test[0])를 파란색 선으로 플롯
# 첫 번째 테스트 샘플의 복원된 데이터(decoded_imgs[0])를 빨간색 선으로 플롯
# 원본 데이터와 복원된 데이터 사이의 차이를 시각화하기 위해 채워진 영역을 추가 (오차 시각화)
# 범례 추가
# 그래프 표시

이전과 마찬가지로 평균 절대 오차에 표준 편차 1을 더한 값에서 임계값을 계산합니다.

In [ ]:
# VAE 모델을 사용하여 정상 학습 데이터(normal_X_train)에 대한 복원 결과(reconstructions) 생성
# 원본 데이터(normal_X_train)와 복원된 데이터(reconstructions) 간의 MAE(Mean Absolute Error) 손실 계산
# 학습 손실(train_loss)의 분포를 히스토그램으로 시각화

In [ ]:
# 학습 손실(train_loss)의 평균과 표준편차를 이용하여 VAE의 이상 탐지 임계값(threshold_vae) 설정
# 설정된 임계값 출력

In [ ]:
# VAE 모델을 사용하여 비정상 테스트 데이터(abnormal_X_test)에 대한 복원 결과 생성
# 원본 데이터(abnormal_X_test)와 복원된 데이터(reconstructions) 간의 MAE(Mean Absolute Error) 손실 계산
# 테스트 손실(test_loss)의 분포를 히스토그램으로 시각화
# 설정된 임계값(threshold_vae)을 빨간색 선으로 표시
# 그래프의 x축과 y축 레이블 설정
# 그래프 표시

재구성 오류가 임계값보다 큰 경우 ECG를 이상으로 분류합니다.

### variational autoencoder를 이용한 예측

### autoencoder를 이용한 예측

RNN 이진 분류 모델